In [2]:
import pandas as pd 
import numpy as np
import matplotlib as plt
import matplotlib.pyplot as plt
import seaborn as sns
import os
# widely used for a variety of image and video analysis
import cv2
import random
import glob

In [4]:
df = pd.read_csv('/Users/cochral/Desktop/SLAEP/Topdown/videos/HOLES/identify-holes/n9-belowaverageremoved.csv')

VIDEO OF LARVAE 

In [5]:

# Set the image size
image_size = 1400

# Define the codec and create VideoWriter object
fourcc = cv2.VideoWriter_fourcc(*'mp4v')  # Codec used to compress the frames
video_output = cv2.VideoWriter('/Users/cochral/Desktop/SLAEP/Topdown/videos/HOLES/identify-holes/n9-removed.mp4', fourcc, 10.0, (image_size, image_size))

df = df.sort_values('frame_idx')

def generate_random_color():
    return (random.randint(0, 255), random.randint(0, 255), random.randint(0, 255))

# Dictionary to store the last index of each track
last_index = {}
# Dictionary to maintain color assignment for each track
track_colors = {}

# Iterate over each frame
for frame in range(df['frame_idx'].min(), df['frame_idx'].max() + 1):
    frame_df = df[df['frame_idx'] == frame]

    # Initialize a blank image for this frame
    frame_image = np.zeros((image_size, image_size, 3), dtype=np.uint8)

    # Determine which tracks are currently active
    current_tracks = set(frame_df['track'].unique())

    # Identify tracks that have disappeared
    disappeared_tracks = set(last_index.keys()) - current_tracks
    for track in disappeared_tracks:
        del last_index[track]  # Remove disappeared tracks from last_index

    # Update the track_colors dictionary for any new tracks
    for track in current_tracks:
        if track not in track_colors:
            track_colors[track] = generate_random_color()

    for track in current_tracks:
        track_df = frame_df[frame_df['track'] == track]
        if track_df.empty:
            continue

        current_index = track_df.index[0]

        if track in last_index:
            # Retrieve the previous frame's data using the stored last index
            prev_index = last_index[track]
            prev_track_df = df.loc[prev_index]

            for part in ['head', 'body', 'tail']:
                prev_x = prev_track_df[f'{part}.x']
                prev_y = prev_track_df[f'{part}.y']
                curr_x = track_df[f'{part}.x'].values[0]
                curr_y = track_df[f'{part}.y'].values[0]

                # Check if any of the coordinates are NaN before drawing
                if not np.isnan([prev_x, prev_y, curr_x, curr_y]).any():
                    # Convert to integers
                    prev_x, prev_y, curr_x, curr_y = map(int, [prev_x, prev_y, curr_x, curr_y])
                    # Draw the line with the corresponding track color
                    cv2.line(frame_image, (prev_x, prev_y), (curr_x, curr_y),
                             color=track_colors[track], thickness=4)

        # Update the last index for this track
        last_index[track] = current_index

    # Write the frame to the video output
    video_output.write(frame_image)

# Release the video writer
video_output.release()




VIDEO OF TRACKS

In [11]:
# initially read the csv file

df = pd.read_csv('/Volumes/lab-windingm/home/users/cochral/AttractionRig/modelling-behaviour/agarose-day4-n10/2024-05-28_13-39-17_td5.tracks.csv')

# definition to generate a random RGB color (3 channels)
   # return a tuple with three integers ranging from 0 to 255
def generate_random_color():
    return (random.randint(0, 255), random.randint(0, 255), random.randint(0, 255))

# Set the image size
   # current resolution of videos 1400 1400
image_size = 1400

# Directory and output settings
output_path = '/Volumes/lab-windingm/home/users/cochral/AttractionRig/modelling-behaviour/agarose-day4-n10/2024-05-28_13-39-17_td5.tracks/output__test.mp4'

# Define the codec and create VideoWriter object
# This object is used to write frames to the output video at 10 frames per second with the defined image size
fourcc = cv2.VideoWriter_fourcc(*'mp4v')  # Codec used to compress the frames
video_output = cv2.VideoWriter(output_path, fourcc, 10.0, (image_size, image_size))

# Assume 'df' is already loaded and sorted by 'frame_idx'
df = df.sort_values('frame_idx')

# Dictionary to store the last index of each track
  # Stores the last index processed for each track to link the current frame with the previous one
last_index = {}
# Dictionary to maintain unique color assignment for each track
track_colors = {}

# Initialize a cumulative image
  # Initializes a zero-valued image array (cumulative_image) that will be used to draw the paths of the tracked objects
  # dont get this as much 
cumulative_image = np.zeros((image_size, image_size, 3), dtype=np.uint8)

# Iterate over each frame
for frame in range(df['frame_idx'].min(), df['frame_idx'].max() + 1):
    # ensure frame is current frame 
    frame_df = df[df['frame_idx'] == frame]

    # Update the track_colors dictionary for any new tracks in the current frame
    for track in frame_df['track'].unique():
        if track not in track_colors:
            track_colors[track] = generate_random_color()

    # iterate over each track

    for track in frame_df['track'].unique():
        # ensure the track is the current track iteration
        track_df = frame_df[frame_df['track'] == track]
        # if there is no data for the track in the current frame it skips the rest of the loop
        if track_df.empty:
            continue

        # Stores the current index of the track data
           # This helps in linking positions between consecutive frames.

        current_index = track_df.index[0]

        # If the track was present in the previous frame (checked using last_index), it retrieves the previous coordinates and draws lines from the previous positions to the current positions for each body part (head, body, tail)

        if track in last_index:
            # Retrieve the previous frame's data using the stored last index
            prev_index = last_index[track]
            prev_track_df = df.loc[prev_index]

            # iterate over each part 
            for part in ['head', 'body', 'tail']:
                prev_x = prev_track_df[f'{part}.x']
                prev_y = prev_track_df[f'{part}.y']
                curr_x = track_df[f'{part}.x'].values[0]
                curr_y = track_df[f'{part}.y'].values[0]

                # Check if any of the coordinates are NaN before drawing
        
                if not np.isnan([prev_x, prev_y, curr_x, curr_y]).any():
                    # Converts coordinates to integers
                    prev_x, prev_y, curr_x, curr_y = map(int, [prev_x, prev_y, curr_x, curr_y])
                    # Draw the line on cumulative_image with the corresponding track color
                    cv2.line(cumulative_image, (prev_x, prev_y), (curr_x, curr_y),
                             color=track_colors[track], thickness=1)

        # Update the last index for this track
          # important for next frame processing 
        last_index[track] = current_index

    # Write the cumulative image to the video output
    video_output.write(cumulative_image)

# Release the video writer
video_output.release()


KeyError: 'frame_idx'

FIRST SCRIPT

Initializes a new frame_image for each frame, which means only the current movements are drawn on this image. The image is reset to black (empty) at the beginning of processing each frame. This approach produces a video where each frame only shows the movements for that specific frame, without retaining the history of previous movements


SECOND SCRIPT

Utilizes a single cumulative_image that persists throughout all frames, accumulating all tracking lines drawn from the start to the end of the video. This method creates a video where each frame shows all previous movements overlayed, forming a continuous trace of each track's path



### WILL MODIFY THIS CODE SUCH THAT I CAN TEST MY ANALYSIS SCRIPTS 

COUNT INSIDE HOLE

In [13]:

df = pd.read_csv('/Volumes/lab-windingm/home/users/cochral/AttractionRig/analysis/side-hole/food/2024-08-27_13-50-20_td3.tracks.feather_hole_data.csv')


mm_to_pixel = ['x_body', 'y_body']
df[mm_to_pixel] = df[mm_to_pixel] * (1032/90)

# Set the im= age size
image_size = 1400

# Define the codec and create VideoWriter object
fourcc = cv2.VideoWriter_fourcc(*'mp4v')  # Codec used to compress the frames
video_output = cv2.VideoWriter('/Volumes/lab-windingm/home/users/cochral/AttractionRig/analysis/side-hole/food/2024-08-27_13-50-20_td3.tracks.feather_hole_data.csv.mp4', fourcc, 25.0, (image_size, image_size))

df = df.sort_values('frame')

# want to iterate over each frame and generate an image 
for frame in range(int(df['frame'].min()), int(df['frame'].max()) + 1):
    frame_df = df[df['frame'] == frame]

    # Initialize a blank image for this frame
    frame_image = np.zeros((image_size, image_size, 3), dtype=np.uint8)

    for i, row in frame_df.iterrows():

        body_x = row['x_body']
        body_y = row['y_body']

        # Skip if coordinates are NaN
        if not np.isnan([body_x, body_y]).any():
            # Convert coordinates to integers
            body_x, body_y = int(body_x), int(body_y)

            # Determine color based on the boolean column (replace 'your_boolean_column' with actual column name)
            if row['moving_outside']:
                color = (255, 0, 0) # RED
            else:
                color = (0, 0, 255) # BLUE

            
            print(f"Frame: {frame}, X: {body_x}, Y: {body_y}, Moving Outside: {row['moving_outside']}, Color: {color}")

                    # Draw the line with the corresponding track color
            cv2.circle(frame_image, (body_x, body_y), radius=4, color=color, thickness=-1)

    # Write the frame to the video output
    video_output.write(frame_image)

# Release the video writer
video_output.release()




Frame: 0, X: 756, Y: 616, Moving Outside: True, Color: (255, 0, 0)
Frame: 0, X: 669, Y: 672, Moving Outside: True, Color: (255, 0, 0)
Frame: 0, X: 721, Y: 696, Moving Outside: True, Color: (255, 0, 0)
Frame: 0, X: 716, Y: 729, Moving Outside: True, Color: (255, 0, 0)
Frame: 0, X: 336, Y: 976, Moving Outside: True, Color: (255, 0, 0)
Frame: 0, X: 772, Y: 664, Moving Outside: True, Color: (255, 0, 0)
Frame: 1, X: 661, Y: 676, Moving Outside: True, Color: (255, 0, 0)
Frame: 1, X: 712, Y: 735, Moving Outside: True, Color: (255, 0, 0)
Frame: 1, X: 768, Y: 668, Moving Outside: True, Color: (255, 0, 0)
Frame: 1, X: 712, Y: 699, Moving Outside: True, Color: (255, 0, 0)
Frame: 1, X: 760, Y: 604, Moving Outside: True, Color: (255, 0, 0)
Frame: 1, X: 336, Y: 977, Moving Outside: False, Color: (0, 0, 255)
Frame: 2, X: 700, Y: 696, Moving Outside: True, Color: (255, 0, 0)
Frame: 2, X: 652, Y: 677, Moving Outside: True, Color: (255, 0, 0)
Frame: 2, X: 704, Y: 740, Moving Outside: True, Color: (255, 

TRYING TO OVERLAY THE VIDEO

this is chat gpt's work way above my head

In [4]:

df = pd.read_csv('/Volumes/lab-windingm/home/users/cochral/AttractionRig/analysis/side-hole/food/2024-08-27_13-50-20_td3.tracks.feather_hole_data.csv')


mm_to_pixel = ['x_body', 'y_body']
df[mm_to_pixel] = df[mm_to_pixel] * (1032/90)

# Set the im= age size
image_size = 1400

# Open the original MP4 video
original_video_path = '/Volumes/lab-windingm/home/users/cochral/AttractionRig/analysis/side-hole/food/2024-08-27_13-50-20_td3.mp4'
original_video = cv2.VideoCapture(original_video_path)

# Define the codec and create VideoWriter object
fourcc = cv2.VideoWriter_fourcc(*'mp4v')  # Codec used to compress the frames
video_output = cv2.VideoWriter('/Volumes/lab-windingm/home/users/cochral/AttractionRig/analysis/side-hole/food/2024-08-27_13-50-20_td3.tracks.hole_overlay-nopostprocessing.mp4', fourcc, 25.0, (image_size, image_size))

df = df.sort_values('frame')

# Iterate over frames in the original video
frame_number = 0
while original_video.isOpened():
    ret, frame = original_video.read()  # Read the next frame from the original video
    if not ret:
        break  # Exit if no more frames

    # Get the data for the current frame from the CSV
    frame_df = df[df['frame'] == frame_number]

    # Overlay the circles from the CSV data onto the frame
    for i, row in frame_df.iterrows():
        body_x = row['x_body']
        body_y = row['y_body']

        # Skip if coordinates are NaN
        if not np.isnan([body_x, body_y]).any():
            # Convert coordinates to integers
            body_x, body_y = int(body_x), int(body_y)

            # Determine color based on the boolean column
            if row['moving_outside']:
                color = (255, 0, 0)  # RED for moving outside
            else:
                color = (0, 0, 255)  # BLUE for not moving outside

            # Overlay the circle on the original video frame
            cv2.circle(frame, (body_x, body_y), radius=4, color=color, thickness=-1)

    # Write the updated frame to the output video
    video_output.write(frame)

    # Move to the next frame
    frame_number += 1


original_video.release()
video_output.release()




chatgpt getting the hole-boundary overlay

In [2]:
import shapely.wkt
import cv2
import numpy as np
import pandas as pd

# Load the CSV
df = pd.read_csv('/Volumes/lab-windingm/home/users/cochral/AttractionRig/analysis/side-hole/food/2024-09-02_03-52-22_td4.tracks.feather_hole_data.csv')

mm_to_pixel = ['x_body', 'y_body']
df[mm_to_pixel] = df[mm_to_pixel] * (1032/90)

# Set the image size
image_size = 1400

# Open the original MP4 video
original_video_path = '/Volumes/lab-windingm/home/users/cochral/AttractionRig/analysis/side-hole/food/2024-09-02_03-52-22_td4.mp4'
original_video = cv2.VideoCapture(original_video_path)

# Define the codec and create VideoWriter object
fourcc = cv2.VideoWriter_fourcc(*'mp4v')  # Codec used to compress the frames
video_output = cv2.VideoWriter('/Volumes/lab-windingm/home/users/cochral/AttractionRig/analysis/side-hole/food/2024-09-02_03-52-22_td4.tracks.hole_overlay-nopostprocessing-holeboundary.mp4', fourcc, 25.0, (image_size, image_size))

df = df.sort_values('frame')

# Read the WKT file
wkt_file_path = '/Volumes/lab-windingm/home/users/cochral/AttractionRig/analysis/side-hole/food/2024-09-02_03-52-22_td4_hole.wkt'
with open(wkt_file_path, 'r') as file:
    wkt_data = file.read()

# Parse the WKT polygon
polygon = shapely.wkt.loads(wkt_data)

# Extract the polygon coordinates
boundary_coords = np.array(polygon.exterior.coords)

# Convert the coordinates from mm to pixels
boundary_coords *= (1032 / 90)

# Iterate over frames in the original video
frame_number = 0
while original_video.isOpened():
    ret, frame = original_video.read()  # Read the next frame from the original video
    if not ret:
        break  # Exit if no more frames

    # Get the data for the current frame from the CSV
    frame_df = df[df['frame'] == frame_number]

    # Overlay the circles from the CSV data onto the frame
    for i, row in frame_df.iterrows():
        body_x = row['x_body']
        body_y = row['y_body']

        # Skip if coordinates are NaN
        if not np.isnan([body_x, body_y]).any():
            # Convert coordinates to integers
            body_x, body_y = int(body_x), int(body_y)

            # Determine color based on the boolean column
            if row['moving_outside']:
                color = (255, 0, 0)  # RED for moving outside
            else:
                color = (0, 0, 255)  # BLUE for not moving outside

            # Overlay the circle on the original video frame
            cv2.circle(frame, (body_x, body_y), radius=4, color=color, thickness=-1)

    # Draw the polygon (hole boundary) on the frame
    boundary_coords_int = boundary_coords.astype(int)
    cv2.polylines(frame, [boundary_coords_int], isClosed=True, color=(0, 255, 0), thickness=2)  # Green boundary

    # Write the updated frame to the output video
    video_output.write(frame)

    # Move to the next frame
    frame_number += 1

# Release video resources
original_video.release()
video_output.release()
